In [1]:
import os
import sys

# 获取当前工作目录
current_dir = os.getcwd()
# 设置项目根目录
project_root = os.path.abspath(os.path.join(current_dir, ".."))
# 将项目根目录添加到sys.path
sys.path.insert(0, project_root)

In [2]:
errors = set()

In [3]:
from db import kg
from pre import html_processor
from utils import splitter
from api import glm,deepseek
from json import JSONDecodeError


def read_file(root_path, root_name):
    data_models = html_processor.parse_xml(root_path + '/nodetree.xml', root_name)
    print(f'要处理的文件数量: {len(data_models)}')
    
    for loop, data_model in enumerate(data_models):
        print(f'loop: {loop}, root: {data_model.root}, name: {data_model.name}, url: {data_model.url}')
        # 按照url去读取文件
        dst_url = root_path + '/documents/' + data_model.url
        content = html_processor.html_to_markdown(dst_url)
        
        if len(content) > 3000:
            docs = splitter.split_markdown(content, 1024)

            for doc in docs:
                content = '# ' + doc.get('h1') + '\n' if doc.get('h1') else ''
                content += '## ' + doc.get('h2') + '\n' if doc.get('h2') else ''
                content += doc.get('content')
                try:
                    graphs = deepseek.extract_knowledge_graph(content)
                except JSONDecodeError:
                    errors.add(dst_url)
                    continue
                kg.store_graph_extracts(graphs)

        try:
            graphs = deepseek.extract_knowledge_graph(content)
        except JSONDecodeError:
            errors.add(dst_url)
            continue
        kg.store_graph_extracts(graphs)


In [4]:
read_file('D:\\Workspace\\aiops2024-challenge-dataset\\emsplus', 'emsplus')

要处理的文件数量: 769
loop: 0, root: emsplus, name: 更新历史, url: 文档包简介/topics/t-0012863095.html
[
    {
        "head": "R1.1",
        "head_type": "版本号",
        "relation": "更新日期",
        "tail": "2022-05-05",
        "tail_type": "日期"
    },
    {
        "head": "R1.1",
        "head_type": "版本号",
        "relation": "更新说明",
        "tail": "更新了INPUTS参数说明",
        "tail_type": "说明"
    },
    {
        "head": "R1.1",
        "head_type": "版本号",
        "relation": "更新位置",
        "tail": "软件安装",
        "tail_type": "位置"
    },
    {
        "head": "R1.1",
        "head_type": "版本号",
        "relation": "更新说明",
        "tail": "新增",
        "tail_type": "说明"
    },
    {
        "head": "R1.1",
        "head_type": "版本号",
        "relation": "更新位置",
        "tail": "资源名称更新",
        "tail_type": "位置"
    },
    {
        "head": "R1.0",
        "head_type": "版本号",
        "relation": "更新日期",
        "tail": "2022-03-25",
        "tail_type": "日期"
    },
    {
        "head": "R1.0",
   

ValueError: Keys cannot be empty

In [ ]:
read_file('D:\\Workspace\\aiops2024-challenge-dataset\\director', 'director')  

In [ ]:
read_file('D:\\Workspace\\aiops2024-challenge-dataset\\rcp', 'rcp')

In [ ]:
read_file('D:\\Workspace\\aiops2024-challenge-dataset\\umac', 'umac')